In [ ]:
%load_ext autoreload
%autoreload 2

# Make a new data release of the current version of the NENA data

The NENA data should reside locally on your system.

By default, we assume it is under `~/github` and then under *org/repo/folder/version*
where

* *org* = `CambridgeSemiticsLab`
* *repo* = `nena_tf`
* *folder* = `tf`
* *version* = `alpha`

You pass *org*, *repo*, *folder*, *version* as parameters.
You can replace the `~/github` by passing a `source` parameter.

The data will be zipped into a file that will be attached to a new release on GitHub.
This zipfile is created by default in `~/Downloads`, but you can override this by passing a `dest` parameter.

We assume the tf data resides under `~/local/data` and we use `~/local/zips` as landing directory for the zip file. 

Note the parameter `3` after `VERSION` in the call of `releaseData()` below.
This indicates which part of the version number should be increased by one.
The parts behind it will be deleted.

Examples:

old version | method | new version
--- | --- | ---
v2.4.6 | 3 | v2.4.7
v2.4.6 | 2 | v2.5
v2.4.6 | 1 | v3
v2.4 | 3 | v2.4.1
v2 | 3 | v2.0.1
v2 | 2 | v2.1

In [ ]:
from tf.parameters import GH
from tf.advanced.repo import releaseData

BACKEND = GH
ORG = "CambridgeSemiticsLab"
REPO = "nena_tf"
FOLDER = "tf"
VERSION = "alpha"
DATA_IN = "~/local/data"
DATA_ZIP = "~/local/zips"

releaseData(GH, ORG, REPO, FOLDER, VERSION, 3, source=DATA_IN, dest=DATA_ZIP)

# Make all search clients for the CambridgeSemiticsLab/nena dataset

Suppose a new release has been made of the NENA corpus data.
Now we want to regenerate its client search apps.

We assume the config data for the apps is in a local directory on the system.

CONFIG_DIR should have the same structure as
[layeredsearch](https://github.com/annotation/app-nena/tree/master/layeredsearch).
You can tweak it, but it should play nice with the client generation process.

We generate the client search data in a local directory on the system.

In [ ]:
import os
from tf.client.make.build import makeSearchClients

APP_DIR = os.path.expanduser("~/local/app-nena")
OUTPUT_DIR = os.path.expanduser("~/local/lsOut")
DATASET = "nena"

In [ ]:
makeSearchClients(DATASET, OUTPUT_DIR, APP_DIR, dataDir="~/local/tfNew")

# Ship all apps for all corpora

From now on we work in a setting where we ship client apps to GitHub pages of *org*`/`*repo*`-search`.

In [ ]:
import collections
import os

from tf.client.make.build import Make

In [ ]:
APPS = (
    ("ETCBC/bhsa", "structure"),
    ("CLARIAH/wp6-missieven", "text"),
    ("CambridgeSemiticsLab/nena_tf", "phono"),
    ("CambridgeSemiticsLab/nena_tf", "fuzzy"),
)

In [ ]:
APPS_BY_DATASET = (
    ("CambridgeSemiticsLab/nena_tf", ("fuzzy", "phono")),
    ("ETCBC/bhsa", ("structure",)),
    ("CLARIAH/wp6-missieven", ("text",)),
)

In [ ]:
for (dataset, apps) in APPS_BY_DATASET:
    nApps = len(apps)
    for app in apps:
        Mk = Make(dataset, app, debugState="off")
        Mk.ship(publish=nApps == 1)
    if nApps > 1:
        Mk = Make(dataset, None, debugState="off")
        Mk.publish()

# Individual apps in debug mode

In [ ]:
Mk = Make(*APPS[1], debugState="on")

# Load data

The Text-Fabric dataset is loaded.

In [ ]:
Mk.loadTf()

In [ ]:
A = Mk.A
api = A.api
Fs = api.Fs
F = api.F
L = api.L
T = api.T

In [ ]:
T.sectionTypes

# Configure

If you changed critical files in the layered search app (`mkdata.py` or `config.yaml`),
run this command to update the configuration inside the maker.

In [ ]:
# do this if you have changed mkdata.py or config.yaml

Mk.config()

# Settings

Generate the settings for the client app, but do not dump them yet to file.
Also the legends are generated here, which might use the loaded data.

In [ ]:
Mk.makeClientSettings()

# Links

Generate links from section nodes to online locations of those sections.

This is done by simply calling the Text-Fabric API.

In [ ]:
Mk.makeLinks()

# Record

Here we call the app-dependent function `record()`, 
which records the corpus data in `recorders` and `accumulators`.

Some layers can use the position data of other layers, and these layers are stored in accumulators.

Layers with their own position data are stored in recorders, they remember the node positions within
the stored material. This is a Text-Fabric device, see [Recorder](https://annotation.github.io/text-fabric/tf/convert/recorder.html).

In [ ]:
Mk.config()
Mk.record()

# Dump data

The corpus texts and positions are derived from the recorders and accumulators, and written to file.

In [ ]:
Mk.config()
Mk.dumpCorpus()

# Dump config

The client settings, generated in an earlier step, are dumped to file, next to the corpus data.

In [ ]:
Mk.dumpConfig()

# Make

The client app is composed as an HTML page with CSS styles and a Javascript program,
and it is moved to the `site` directory of the repo.

We also set the debug flag reflecting how we initialized the maker: debug is on.

In [ ]:
Mk.makeClient()
Mk.adjustDebug()